In [4]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017, username='root', password='example')

db = client['posts']
posts_collection = db['posts']
posts_collection.drop()




In [5]:
from itertools import islice
import json
from datetime import datetime

with open ('data.jsonl', 'r') as file:
    for line in file:
        data = json.loads(line)
        data['post']['timestamp'] = datetime.fromisoformat(data['post']['timestamp'])
        for comment in data['comments']:
            comment['comment']['timestamp'] = datetime.fromisoformat(comment['comment']['timestamp'])
        posts_collection.insert_one(data)

In [ ]:
#1.1
import json
import random

from faker import Faker

# Create an instance of the Faker class
fake = Faker()

# Define the fields for posts, comments, and user information
post_fields = ["title", "content"]
comment_fields = ["text", "timestamp"]
user_fields = ["name", "email", "age"]

# Generate a list of 5 users with random information
users = []
for i in range(5):
    user = {}
    user["name"] = fake.name()
    user["email"] = fake.email()
    user["age"] = random.randint(18, 60)
    users.append(user)

# Generate random posts and comments for each user
data = []
for user in users:
    num_posts = random.randint(1, 100)
    for _ in range(num_posts):
        post = {}
        post["user"] = user["name"]
        post["user_email"] = user["email"]
        post["user_age"] = user["age"]
        post["post"] = {}
        post["post"]["title"] = f'Random title for {user["name"]}'
        post["post"]["content"] = f'Random content for {user["name"]} {fake.text()}'
        post["post"]["timestamp"] = fake.date_time().isoformat()
        num_comments = random.randint(0, 5)
        post["comments"] = []
        for _ in range(num_comments):
            comment = {}
            comment["user"] = random.choice(users)["name"]
            comment["user_email"] = random.choice(users)["email"]
            comment["user_age"] = random.choice(users)["age"]
            comment["comment"] = {}
            comment["comment"]["text"] = f"Random text {fake.text()}"
            comment["comment"]["timestamp"] = fake.date_time().isoformat()
            post["comments"].append(comment)
        data.append(post)

for item in data:
    posts_collection.insert_one(item)
    print(item)
    print('\n)')

    

In [ ]:
#1.2


import pandas as pd
# Buscar todos los usuarios mayores a 25

users = posts_collection.aggregate([
    {"$match": {"user_age": {"$gt": 25}}},
    {"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
    {"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])


# Convertir el cursor a un DataFrame de pandas
df = pd.DataFrame(list(users))
df


In [6]:
#1.3
# Buscar un usuario por direccion de correo

user = posts_collection.find_one({"user_email": "tmartin@example.org"})
# Imprimir el resultado
print(user)


{'_id': ObjectId('65ed0d6aa4fe388ae296fce5'), 'user': 'Joann Bowen', 'user_email': 'tmartin@example.org', 'user_age': 19, 'post': {'title': 'Random title for Joann Bowen', 'content': 'Random content for Joann Bowen Wide take drop. Down goal still phone chance tonight. See stock beat significant right.\nCommercial eye meet impact risk. Every laugh true leave chair part because.', 'timestamp': datetime.datetime(1978, 9, 14, 0, 1, 17, 95000)}, 'comments': [{'user': 'Joshua Murray', 'user_email': 'tmartin@example.org', 'user_age': 51, 'comment': {'text': 'Random text Forget blue different most drive amount change. Beat successful inside court guess she thank. Image serious report cell federal majority how agent.', 'timestamp': datetime.datetime(1971, 3, 8, 2, 26, 36, 557000)}}]}


In [ ]:
#1.4

In [ ]:
#1.5

In [ ]:
#2.1
# Buscar todos los usuarios cuyo nombre de usuario contenga la palabra S̈mith.̈ (Expresiones regulares)
users = posts_collection.aggregate([
    {"$match": {"user": {"$regex": "Smith", "$options": "i"}}},
    {"$group": {"_id": "$user"}},
    {"$project": {"_id": 0, "user": "$_id"}}
])


# Convertir el cursor a un DataFrame de pandas
df = pd.DataFrame(list(users))
df


In [ ]:
#2.2
# Buscar publicaciones creadas después de una fecha específica (por ejemplo, 2000-01-01)


date = datetime(2000, 1, 1)


posts = posts_collection.find(
    {"post.timestamp": {"$gt": date}},
    {"user": 1, "post.timestamp": 1, "_id": 0} # Seleccionar solo el nombre del usuario y la fecha
)


df = pd.DataFrame(list(posts))
df.rename(columns={"post": "timestamp"}, inplace=True)  # Renombrar la columna "post" a "timestamp"
df


In [ ]:
#2.3

In [ ]:
#2.4

In [ ]:
#2.5

In [ ]:
#3.1
#Buscar usuarios que tengan más de 20 años pero menos de 30. (Combinar operadores)

users = posts_collection.aggregate([
{"$match": {"user_age": {"$gt": 25, "$lt": 30}}},
{"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
{"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])

df = pd.DataFrame(list(users))
df

In [ ]:
#3.2

# Recuperar solo el nombre de usuario y la edad del usuario de todos los documentos. (Proyección
users = posts_collection.find({}, {"user": 1, "user_age": 1, "_id": 0})
df = pd.DataFrame(list(users))
df

In [ ]:
#3.3

In [ ]:
#3.4

In [ ]:
#3.5

In [ ]:
#4.1
# Ordenar todos los usuarios por edad en orden descendente

users = posts_collection.aggregate([
{"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
{"$sort": {"user_age": -1}},
{"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])

df = pd.DataFrame(list(users))
df

In [25]:
#4.2

#Buscar los top 3 usuarios principales con las publicaciones más recientes (en función de la marca de tiempo).

users = posts_collection.aggregate([
    {"$sort": {"post.timestamp": -1}},
    {"$group": {"_id": "$user", "latest_post": {"$first": "$post.timestamp"}}},
    {"$sort": {"latest_post": -1}},
    {"$limit": 3},
    {"$project": {"_id": 0, "user": "$_id", "latest_post": 1}}
])

df = pd.DataFrame(list(users))
df

,latest_post,user
0,2024-02-28 20:30:48.385,Michael Smith
1,2024-02-26 09:53:04.022,Erin Smith
2,2024-02-04 15:56:25.075,Gary Best


In [ ]:
#4.3

In [ ]:
#5.1
#1. Utilizar el pipeline de agregación para calcular el número total de usuarios.

users = posts_collection.aggregate([
{"$group": {"_id": "$user"}},
{"$group": {"_id": None, "Cantidad": {"$sum": 1}}},
{"$project": {"_id": 0, "Cantidad": 1}}
])

df = pd.DataFrame(list(users))
df


In [32]:
#5.2

#  Encontrar al usuario con el contenido de publicación más largo (en función del número de caracteres)

users = posts_collection.aggregate([
    {"$project": {"user": 1, "largo_Contenido": {"$strLenCP": "$post.content"}}},
    {"$sort": {"largo_Contenido": -1}},
     {"$limit": 1}
])

df = pd.DataFrame(list(users))
df

,_id,user,largo_Contenido
0,65ed0d6ca4fe388ae29700b0,Christopher Jenkins,238


In [ ]:
#5.3

In [ ]:
#5.4

In [ ]:
#5.5

In [ ]:
#6.1

# Incrementar la edad de todos los usuarios en 1 con operadores

posts_collection.update_many({}, {"$inc": {"user_age": 1}})

users = posts_collection.aggregate([
    {"$group": {"_id": "$user", "user_age": {"$first": "$user_age"}}},
    {"$project": {"_id": 0, "user": "$_id", "user_age": 1}}
])


df = pd.DataFrame(list(users))
df

In [50]:
#6.2

# Establecer el campo ”contenido” de una publicación específica en una cadena vacía.

posts_collection.update_one({"post.title": "Random title for Mrs. Christine Sawyer"}, {"$set": {"post.content": ""}})

post = posts_collection.find({"post.content": ""})

df = pd.DataFrame(list(post))
df

,_id,user,user_email,user_age,post,comments
0,65ed0d6aa4fe388ae296fce5,Joann Bowen,tmartin@example.org,19,"{'title': 'Random title for Joann Bowen', 'con...","[{'user': 'Joshua Murray', 'user_email': 'tmar..."
1,65ed0d6aa4fe388ae296fcf1,Mrs. Christine Sawyer,edwardsmeagan@example.com,37,{'title': 'Random title for Mrs. Christine Saw...,"[{'user': 'Christopher Jenkins', 'user_email':..."
2,65ed0d6aa4fe388ae296fcf6,Kayla Hill,snyderkathy@example.net,29,"{'title': 'Random title for Kayla Hill', 'cont...","[{'user': 'Jon Rodriguez', 'user_email': 'jose..."


In [ ]:
#6.3

In [ ]:
#6.4

In [ ]:
#7.1

# Crear un índice en el campo ”user_email” para búsquedas de correo electrónico más rápidas.
posts_collection.create_index([("user_email", 1)])

df = pd.DataFrame(list(posts_collection.index_information()))
df


In [53]:
#7.2

# Explicar el plan de consulta para encontrar usuarios con edad mayor a 30.


posts_collection.find({"user_age": {"$gt": 30}}).explain()







{'explainVersion': '1',
 'queryPlanner': {'namespace': 'posts.posts',
  'indexFilterSet': False,
  'parsedQuery': {'user_age': {'$gt': 30}},
  'queryHash': 'A95D609D',
  'planCacheKey': 'A95D609D',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'COLLSCAN',
   'filter': {'user_age': {'$gt': 30}},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 860,
  'executionTimeMillis': 1,
  'totalKeysExamined': 0,
  'totalDocsExamined': 1435,
  'executionStages': {'stage': 'COLLSCAN',
   'filter': {'user_age': {'$gt': 30}},
   'nReturned': 860,
   'executionTimeMillisEstimate': 0,
   'works': 1436,
   'advanced': 860,
   'needTime': 575,
   'needYield': 0,
   'saveState': 1,
   'restoreState': 1,
   'isEOF': 1,
   'direction': 'forward',
   'docsExamined': 1435},
  'allPlansExecution': []},
 'command': {'find': 'posts',
  'filter': {'use

In [ ]:
#7.3